# --- Import packages and create a spark session ---

In [43]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, when, count, col, regexp_extract, avg, round
import plotly.graph_objects as go
import plotly.express as px

#### Create a SparkSession

In [2]:
spark = SparkSession.builder.appName("titanic_spark").getOrCreate()

24/04/08 15:22:57 WARN Utils: Your hostname, Tiens-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.20.17 instead (on interface en0)
24/04/08 15:22:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/08 15:22:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 49797)
Traceback (most recent call last):
  File "/Users/tienpham/.pyenv/versions/3.9.13/lib/python3.9/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Users/tienpham/.pyenv/versions/3.9.13/lib/python3.9/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/Users/tienpham/.pyenv/versions/3.9.13/lib/python3.9/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/tienpham/.pyenv/versions/3.9.13/lib/python3.9/socketserver.py", line 747, in __init__
    self.handle()
  File "/Volumes/Data/Study/Data Engineer/Sources/Titanic/.venv/lib/python3.9/site-packages/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/Volumes/Data/Study/Data Engineer/Sources/Titanic/.venv/lib/python3.9/s

# --- Load data ---

#### Read the datasets

In [3]:
train = spark.read.csv("./data/train.csv", header=True, inferSchema=True)
test = spark.read.csv("./data/test.csv", header=True, inferSchema=True)

In [ ]:
train.show(5)

In [ ]:
test.show(5)

#### Explain the dataset
Survived: 0 (No) - 1 (Yes)

Pclass: 1 - 2 - 3 (Ticket class)

SibSp: (Number of siblings/spouses aboard the Titanic)

Parch: (Number of childer aboard the Titanic)

Ticket: (Ticket number)

Fare: (Passenger fare)

Cabin: (Cabin number)

Embarked: C (Cherbourg) - S (Southampton) - Q (Queenstown) (Port of embarkation)

# --- Preprocess data ---

### 1. Preprocess for the train dataset

#### Summary statistics for the train dataset

In [ ]:
train.describe().toPandas()

In [ ]:
train.printSchema()

#### Count the number of missing values in each column

In [ ]:
train.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in train.columns]).toPandas()

#### Check the total number of rows in the train dataset

In [ ]:
train.count()

#### There are a lot of NULL values in the Cabin column, so I decided to drop this column.

In [4]:
train = train.drop("Cabin")

#### Check NULL values for the Embarked column in the train dataset

In [ ]:
train.where(col("Embarked").isNull()).show()

#### Find the most frequent value for the Embarked column in the train dataset

In [ ]:
train.groupBy("Embarked").count().show()

#### Replace the NULL value in the Embarked column with the most frequent value

In [5]:
train = train.na.fill("S", subset=["Embarked"])

#### Find the title in the Name column

In [6]:
train = train.withColumn("Title", regexp_extract(col("Name"), "([A-Za-z]+)\.", 1))
train.groupBy("Title").count().show()

+--------+-----+
|   Title|count|
+--------+-----+
|     Don|    1|
|    Miss|  182|
|Countess|    1|
|     Col|    2|
|     Rev|    6|
|    Lady|    1|
|  Master|   40|
|     Mme|    1|
|    Capt|    1|
|      Mr|  517|
|      Dr|    7|
|     Mrs|  125|
|     Sir|    1|
|Jonkheer|    1|
|    Mlle|    2|
|   Major|    2|
|      Ms|    1|
+--------+-----+



#### Check each Title to find the appropriate Title for replacing

In [ ]:
train.where(col("Title") == 'Don').show()

#### -> Change 'Don' title to 'Mr'

In [ ]:
train.where(col("Title") == 'Countess').show()

#### -> Change 'Countess' title to Ms

In [ ]:
train.where(col("Title") == 'Col').show()

#### -> Change 'Col' title to 'Mr'

In [ ]:
train.where(col("Title") == 'Rev').show()

#### -> Change 'Rev' title to 'Mr'

In [ ]:
train.where(col("Title") == 'Mme').show()

#### -> Change 'Mme' title to 'Ms'

In [ ]:
train.where(col("Title") == 'Capt').show()

#### -> Change 'Capt' title to 'Mr'

In [ ]:
train.where(col("Title") == 'Jonkheer').show()

#### -> Change 'Jonkheer' title to 'Mr'

In [ ]:
train.where(col("Title") == 'Mlle').show()

#### -> Change 'Mlle' title to 'Ms'

In [ ]:
train.where(col("Title") == 'Major').show()

#### -> Change 'Major' title to 'Mr'

#### Replace misspelling title values

In [7]:
train = train.replace(["Don", "Countess", "Col", "Rev", "Mme", "Capt", "Jonkheer", "Mlle", "Major", "Sir", "Lady"], 
                      ["Mr", "Ms", "Mr", "Mr", "Ms", "Mr", "Mr", "Ms", "Mr", "Mr", "Ms"], 
                      "Title")
train.groupBy("Title").count().show()

+------+-----+
| Title|count|
+------+-----+
|  Miss|  182|
|Master|   40|
|    Mr|  531|
|    Dr|    7|
|   Mrs|  125|
|    Ms|    6|
+------+-----+



#### Check the Title values matching with the Sex values

In [ ]:
train.groupBy(["Title", "Sex"]).count().show()

#### Calculate the average age for each title

In [8]:
avg_age_df = spark.createDataFrame(train.groupBy(["Title"]).agg(avg("Age").alias("Avg_Age")).toPandas())
avg_age_df = avg_age_df.withColumn("Avg_Age", round(avg_age_df["Avg_Age"]).cast("double"))
avg_age_df.show()

+------+-------+
| Title|Avg_Age|
+------+-------+
|  Miss|   22.0|
|Master|    5.0|
|    Mr|   33.0|
|    Dr|   42.0|
|   Mrs|   36.0|
|    Ms|   30.0|
+------+-------+



#### Replace the NULL values in the Age column with the average values for each Title

In [9]:
for row in avg_age_df.collect():
    train = train.withColumn("Age", when((col("Title") == row["Title"]) & (col("Age").isNull()), row["Avg_Age"]).otherwise(col("Age")))

### Check NULL values in the train dataset

In [ ]:
train.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in train.columns]).toPandas()

#### Remove unnecessary columns for the analysis

In [10]:
train = train.drop("PassengerId", "Name", "Ticket", "Title")
train.show(5)

+--------+------+------+----+-----+-----+-------+--------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|
+--------+------+------+----+-----+-----+-------+--------+
|       0|     3|  male|22.0|    1|    0|   7.25|       S|
|       1|     1|female|38.0|    1|    0|71.2833|       C|
|       1|     3|female|26.0|    0|    0|  7.925|       S|
|       1|     1|female|35.0|    1|    0|   53.1|       S|
|       0|     3|  male|35.0|    0|    0|   8.05|       S|
+--------+------+------+----+-----+-----+-------+--------+
only showing top 5 rows



### Preprocess for the test dataset

In [ ]:
test.describe().toPandas()

#### Count the number of missing values in each column

In [ ]:
test.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in test.columns]).toPandas()

#### Check the total number of rows in the test dataset

In [ ]:
test.count()

#### Remove the Cabin column due to lots of missing values

In [11]:
test = test.drop("Cabin")
test.show(5)

+-----------+------+--------------------+------+----+-----+-----+-------+-------+--------+
|PassengerId|Pclass|                Name|   Sex| Age|SibSp|Parch| Ticket|   Fare|Embarked|
+-----------+------+--------------------+------+----+-----+-----+-------+-------+--------+
|        892|     3|    Kelly, Mr. James|  male|34.5|    0|    0| 330911| 7.8292|       Q|
|        893|     3|Wilkes, Mrs. Jame...|female|47.0|    1|    0| 363272|    7.0|       S|
|        894|     2|Myles, Mr. Thomas...|  male|62.0|    0|    0| 240276| 9.6875|       Q|
|        895|     3|    Wirz, Mr. Albert|  male|27.0|    0|    0| 315154| 8.6625|       S|
|        896|     3|Hirvonen, Mrs. Al...|female|22.0|    1|    1|3101298|12.2875|       S|
+-----------+------+--------------------+------+----+-----+-----+-------+-------+--------+
only showing top 5 rows



#### Check the row with NULL value in the Fare column for the test dataset

In [ ]:
test.where(col("Fare").isNull()).show()

#### Replace the NULL value in the Fare column with the average value for Pclass=3, Sex=male, and Embarked=S for the test dataset

In [12]:
avg_fare = test.filter("Pclass=3 and Sex='male' and Embarked='S'").agg({"Fare": "avg"}).collect()[0][0]
test = test.na.fill(avg_fare, subset=["Fare"])

In [ ]:
test.where(col("Fare").isNull()).show()

#### Find the title in the Name column

In [13]:
test = test.withColumn("Title", regexp_extract(col("Name"), "([A-Za-z]+)\.", 1))
test.groupBy("Title").count().show()

+------+-----+
| Title|count|
+------+-----+
|  Dona|    1|
|  Miss|   78|
|   Col|    2|
|   Rev|    2|
|Master|   21|
|    Mr|  240|
|    Dr|    1|
|   Mrs|   72|
|    Ms|    1|
+------+-----+



In [ ]:
test.where(col("Title") == 'Dona').show()

#### -> Change the 'Dona' title to 'Ms'

In [ ]:
test.where(col("Title") == 'Col').show()

#### -> Change the 'Col' title to 'Mr'

In [ ]:
test.where(col("Title") == 'Rev').show()

#### -> Change the 'Rev' title to 'Mr'

In [ ]:
test.where(col("Title") == 'Dr').show()

#### -> Change the 'Dr' title to 'Mr'

#### Replace misspelling title values

In [14]:
test = test.replace(["Dona", "Col", "Rev", "Dr"], 
                    ["Ms", "Mr", "Mr", "Mr"], 
                    "Title")
test.groupBy("Title").count().show()

+------+-----+
| Title|count|
+------+-----+
|  Miss|   78|
|Master|   21|
|    Mr|  245|
|   Mrs|   72|
|    Ms|    2|
+------+-----+



#### Check the Title values matching the the Sex values

In [ ]:
test.groupBy(["Title", "Sex"]).count().show()

#### Calculate the average values for each title

In [15]:
avg_age_df_test = spark.createDataFrame(test.groupBy(["Title"]).agg(avg("Age").alias("Avg_Age")).toPandas())
avg_age_df_test = avg_age_df_test.withColumn("Avg_Age", round(avg_age_df_test["Avg_Age"]).cast("double"))
avg_age_df_test.show()

+------+-------+
| Title|Avg_Age|
+------+-------+
|  Miss|   22.0|
|Master|    7.0|
|    Mr|   32.0|
|   Mrs|   39.0|
|    Ms|   39.0|
+------+-------+



#### Replace the NULL values in the Age column with the average age for each title

In [16]:
for row in avg_age_df_test.collect():
    test = test.withColumn("Age", when((col("Title") == row["Title"]) & (col("Age").isNull()), row["Avg_Age"]).otherwise(col("Age")))

#### Check NULL values in the test dataset

In [ ]:
test.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in test.columns]).toPandas()

#### Remove unnecessary columns for the analysis

In [17]:
test = test.drop("PassengerId", "Name", "Ticket", "Title")
test.show(5)

+------+------+----+-----+-----+-------+--------+
|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|
+------+------+----+-----+-----+-------+--------+
|     3|  male|34.5|    0|    0| 7.8292|       Q|
|     3|female|47.0|    1|    0|    7.0|       S|
|     2|  male|62.0|    0|    0| 9.6875|       Q|
|     3|  male|27.0|    0|    0| 8.6625|       S|
|     3|female|22.0|    1|    1|12.2875|       S|
+------+------+----+-----+-----+-------+--------+
only showing top 5 rows



# --- Exploratory Data Analysis ---

In [18]:
df_train = train.toPandas()

#### 1. Explore feature 'Pclass'

In [83]:
df_pclass = df_train.groupby(['Pclass', 'Survived']).size().reset_index()
df_pclass.rename(columns={0: 'Count'}, inplace=True)
df_pclass['Percentage'] = df_train.groupby(['Pclass', 'Survived']).size().groupby(level=0).apply(lambda x: 100 * x / float(x.sum())).values
df_pclass['PclassLabel'] = np.where(df_pclass['Pclass'] == 1, 'Class 1', np.where(df_pclass['Pclass'] == 2, 'Class 2', 'Class 3'))
df_pclass['SurvivedLabel'] = np.where(df_pclass['Survived'] == 0, 'Not Survived', 'Survived')

fig = px.bar(df_pclass, x = 'PclassLabel', y = ['Count'], color='SurvivedLabel', 
             text = df_pclass['Percentage'].apply(lambda x: '{0:1.2f}%'.format(x)), barmode='group',
             color_discrete_sequence=["#bd7ebe", "seagreen"],)
fig.update_layout(title={
                    'text': "Survival Rate by Ticket Class",
                    'y':0.95,
                    'x':0.5,
                    'xanchor': 'center',
                    'yanchor': 'top'}, 
                legend_title_text='',
                barmode='group',
                autosize=False,
                width=700,
                height=500,
                paper_bgcolor="LightSteelBlue",
                xaxis_title="",
                yaxis_title="")
fig.show()

##### Class 3 had the highest number of passengers. It also indicated that the number of passengers died in Class 3 was significantly higher than other classes. Class 1 had the highest number of survival passengers followed by Class 3 and Class 2.

#### 2. Explore features 'SibSp'

In [92]:
survival_rates = df_train.groupby('SibSp')['Survived'].mean().reset_index()
fig = px.bar(survival_rates, x='SibSp', y='Survived', 
            labels={'Survived': 'Survived Probability'})
fig.update_layout(title={
                    'text': "Survived Probability by number of Siblings or Spouse",
                    'y':0.95,
                    'x':0.5,
                    'xanchor': 'center',
                    'yanchor': 'top'}, 
                autosize=False,
                width=700,
                height=500,
                paper_bgcolor="LightSteelBlue",
                xaxis=dict(
                    title = "Number of Siblings or Spouse",
                    type='category')
)
fig.show()

##### There was a maximum of 8 siblings and spouses traveled along with one of the passenger. With passengers having one sibling or spouse, the survived probability was more than 50%. Passengers traveling with 2 siblings or spouse had the survived probability nearly 50% with the value at 46%. The chances of survival dropped drastically if someone traveled with more than 2 siblings or spouse.

#### 3. Explore feature 'Parch'

In [97]:
survival_rates = df_train.groupby('Parch')['Survived'].mean().reset_index()
fig = px.bar(survival_rates, x='Parch', y='Survived', 
            labels={'Survived': 'Survived Probability'})
fig.update_layout(title={
                    'text': "Survived Probability by number of Parent or Children",
                    'y':0.95,
                    'x':0.5,
                    'xanchor': 'center',
                    'yanchor': 'top'}, 
                autosize=False,
                width=700,
                height=500,
                paper_bgcolor="LightSteelBlue",
                xaxis=dict(
                    title = "Number of Parent or Children",
                    type='category')
)
fig.show()

##### Passengers traveled with 1 to 3 Parent or Children had the chance of survival more than 50%, especially with the highest probability for 3 Parent or Children at 60%. With passenger traveled alone, the chance of survival was 34% while the survived probability for 5 Parent or Children was 20%. There was no chance of survival for passengers traveled with 4 or 6 Parent or Children.

#### 4. Explore feature 'Fare'

In [107]:
df_fare = df_train.groupby(['Fare', 'Survived']).size().reset_index()
df_fare['SurvivedLabel'] = np.where(df_fare['Survived'] == 0, 'Not Survived', 'Survived')
df_fare.rename(columns={0: 'Count'}, inplace=True)
fig = px.histogram(df_fare, x="Fare", y="Count", color="SurvivedLabel",
                   marginal="box", color_discrete_sequence=["#bd7ebe", "seagreen"])
fig.update_layout(title={
                    'text': "Survived Distribution by Fare",
                    'y':0.95,
                    'x':0.5,
                    'xanchor': 'center',
                    'yanchor': 'top'}, 
                legend_title_text='',
                autosize=False,
                width=700,
                height=500,
                paper_bgcolor="LightSteelBlue",
                yaxis=dict(
                    title = "Total Passengers")
)
fig.show()


In [31]:
not_survived = df_train[df_train['Survived'] == 0]
survived = df_train[df_train['Survived'] == 1]
pclass = sorted(df_train['PclassCat'].unique().tolist())
fig = go.Figure()
fig.add_trace(go.Bar(x=pclass, y=survived.groupby('Pclass').count().reset_index()['Survived'], name='Survived'))
fig.add_trace(go.Bar(x=pclass, y=not_survived.groupby('Pclass').count().reset_index()['Survived'], name='Not Survived'))
fig.update_layout(title={
                    'text': "Survival Rate by Class",
                    'y':0.9,
                    'x':0.5,
                    'xanchor': 'center',
                    'yanchor': 'top'}, 
                barmode='group',
                autosize=False,
                width=600,
                height=400,
                paper_bgcolor="LightSteelBlue",)
fig.show()